In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import sqlite3
import glob

import numpy as np
import xgboost
import json
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import StratifiedShuffleSplit
from collections import Counter
print(sqlite3.sqlite_version)
import optuna
from catboost import CatBoostClassifier

3.37.2


# load data

In [3]:
load_final_panel=1

In [4]:
%%time
path = '/kaggle/input/panel-with-new-features-v3/'

if load_final_panel:
    print('load final panel')
    print('read targets')
    train_target = pd.read_parquet(path+'train_target', engine="pyarrow")
    test_target = pd.read_parquet(path+'test_target', engine="pyarrow")
    print('read df')
    df_train = pd.read_parquet(path+'df_train_added_features_final', engine="pyarrow")
    df_test  = pd.read_parquet(path+'df_test_added_features_final', engine="pyarrow")

    new_feat_path = '/kaggle/input/new-features-weekly-pivoted/new_features_df_set2'

    pivoted_weekly_features  = pd.read_parquet(new_feat_path, engine="pyarrow")
    
    df_train = df_train.merge(pivoted_weekly_features,on='Device_ID',how='left')
    df_test = df_test.merge(pivoted_weekly_features,on='Device_ID',how='left')

    df_train_columns = list(df_train.columns)

    print(df_train.isna().any().sum(),'train_na')
    print(df_test.isna().any().sum(),'test_na')

    print(df_train.shape)

    print('end')

load final panel
read targets
read df
0 train_na
0 test_na
(5538, 106366)
end
CPU times: user 37 s, sys: 5.69 s, total: 42.7 s
Wall time: 23.1 s


In [5]:
df_train.sort_values(by = ["Device_ID"], inplace = True)
df_train.reset_index(drop=True, inplace=True)

train_target.sort_values(by = ["Device_ID"], inplace = True)
train_target.reset_index(drop=True, inplace=True)

df_train_Device_ID = df_train["Device_ID"]
df_train.drop(columns = ["Device_ID"], inplace = True)
train_target.drop(columns = ["Device_ID"], inplace = True)

train_target['Target'] = train_target.Target.astype('category')

In [6]:
df_test.sort_values(by = ["Device_ID"], inplace = True)
df_test.reset_index(drop=True, inplace=True)

test_target.sort_values(by = ["Device_ID"], inplace = True)
test_target.reset_index(drop=True, inplace=True)

df_test_Device_ID = df_test["Device_ID"]
df_test.drop(columns = ["Device_ID"], inplace = True)
test_target.drop(columns = ["Device_ID"], inplace = True)

test_target['Target'] = test_target.Target.astype('category')

In [7]:
print('data loaded !!!!!!!!!!!!!!!!!!!!')

data loaded !!!!!!!!!!!!!!!!!!!!


In [8]:
from sklearn.model_selection import train_test_split
USE_CALIBRATED_SEED = True
RANDOM_SPLIT_SEED = 1062

if USE_CALIBRATED_SEED:

    df_all = pd.concat([df_train,df_test])
    target_all = pd.concat([train_target,test_target])
    
    df_train, df_test, train_target, test_target = train_test_split(df_all, target_all,
                                                            test_size=0.1, random_state=1062)
    print('data resplited')


data resplited


In [9]:
print('start modeling')

start modeling


In [10]:
use_final_columns = 0
use_rfe = not use_final_columns
save_results = 1
single_run = 1
cv = 0

CATBOOST_OPTUNA = 0
XGBOOST_OPTUNA = 1


In [11]:
MODEL_NAME_A = 'XGB_model_V25_weekly_features.json'
COLUMNS_NAME = "best_features_V25.json"

In [12]:
if use_final_columns:
    with open('/kaggle/input/best-features-0-8905/best_features_0.8905.json', "r") as fp:
        best_feat= json.load(fp)


    best_feat = list(set(best_feat + w3 + best_88_of_all+
                         best_88_of_all_step_20K+best_88_of_all_step_10K+best_100_all_step_30+
                       best_88_all_step_30 + best_88_of_not_it_saved ))


    df_train = df_train[best_feat]
    df_test = df_test[best_feat]
    print(df_train.shape)

In [13]:
optuna_params = {'n_estimators': 1128, 'max_depth': 4, 'min_child_weight': 3, 'learning_rate': 0.02849236407842813, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0.6925413932342802, 'reg_lambda': 0.009751526420604062, 'gamma': 0.7000000000000001}

# RFE

In [14]:
%%time


if use_rfe :
    print('start')

    if optuna_params :
        print('params : ',optuna_params)

    else:
    
        optuna_params = {'n_estimators': 300, 'max_depth': 9, 'min_child_weight': 4, 
                         'learning_rate': 0.023978413719368537, 'subsample': 0.8,
                         'colsample_bytree': 0.7000000000000001, 'reg_alpha': 0.042291352379944136,
                         'reg_lambda': 0.25851748434291855, 'gamma': 0.7}


    
    model_params = {'random_state':17, 'n_jobs':-1, 'tree_method':"hist", 
     'device' : "cuda",  'objective' :'binary:logistic' ,'eval_metric' :roc_auc_score}

    params = optuna_params | model_params


    xgb_reg = xgboost.XGBRegressor(**params)

    selector = RFE(xgb_reg, n_features_to_select=1500, step=30000, verbose=100)
    print('rfe completed')
    selector = selector.fit(df_train, train_target)
    print('fit completed')

    best_features = list(df_train.columns[selector.support_])

    train_prediction = selector.estimator_.predict(df_train[best_features])
    print(f'The auc for train set: {round(roc_auc_score(train_target.Target.astype(int),train_prediction), 3)}')

    test_prediction = selector.estimator_.predict(df_test[best_features])
    print(f'The auc for validation set: {round(roc_auc_score(test_target.Target,test_prediction), 3)}')

    first_words = [s.split('_')[0] for s in best_features]
    word_counts = Counter(first_words)
    print(word_counts)

start
params :  {'n_estimators': 1128, 'max_depth': 4, 'min_child_weight': 3, 'learning_rate': 0.02849236407842813, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0.6925413932342802, 'reg_lambda': 0.009751526420604062, 'gamma': 0.7000000000000001}
rfe completed
Fitting estimator with 106365 features.
Fitting estimator with 76365 features.
Fitting estimator with 46365 features.
Fitting estimator with 16365 features.
fit completed


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [20:14:55] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


The auc for train set: 1.0
The auc for validation set: 0.874
Counter({'Domain': 1215, 'cls': 102, 'unique': 73, 'total': 36, '24': 14, '7': 13, '30': 13, 'ts': 8, 'sum': 8, 'acf': 4, 'smoothing': 4, 'diff': 3, 'repeating': 3, 'max': 1, 'mean': 1, 'std': 1, 'enthropy': 1})
CPU times: user 9min 7s, sys: 18.4 s, total: 9min 25s
Wall time: 4min 35s


In [15]:
if use_rfe:    
    df_train = df_train[best_features]
    df_test = df_test[best_features]

# train model

In [16]:
%%time

from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer


if single_run:

    if optuna_params:
        print('params : ',optuna_params)

    else:
        optuna_params = \
    {'n_estimators': 862, 'max_depth': 8, 'min_child_weight': 5, 'learning_rate': 0.031721261460108965, 'subsample': 1.0, 'colsample_bytree': 0.2, 'reg_alpha': 0.10799432879864407, 'reg_lambda': 0.6650032452501936, 'gamma': 0.7}

    model_params = {'random_state':17, 'n_jobs':-1, 'tree_method':"hist", 
     'device' : "cuda",  'objective' :'binary:logistic' ,'eval_metric' :roc_auc_score}

    params = optuna_params | model_params

    
    xgb_reg = xgboost.XGBRegressor(**params)
    xgb_reg.fit(df_train, train_target.Target.astype(int))
    
    train_prediction = xgb_reg.predict(df_train)
    print(f'The auc for train set: {round(roc_auc_score(train_target.Target.astype(int),train_prediction), 3)}')

    print(df_train.shape)
    test_prediction = xgb_reg.predict(df_test)
    print(f'The auc for validation set: {round(roc_auc_score(test_target.Target.astype(int),test_prediction), 3)}')

    if save_results:
        xgb_reg.save_model(MODEL_NAME_A)
        with open(COLUMNS_NAME, "w") as fp:
            json.dump(list(df_train.columns), fp)

params :  {'n_estimators': 1128, 'max_depth': 4, 'min_child_weight': 3, 'learning_rate': 0.02849236407842813, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0.6925413932342802, 'reg_lambda': 0.009751526420604062, 'gamma': 0.7000000000000001}
The auc for train set: 1.0
(5538, 1500)
The auc for validation set: 0.874
CPU times: user 10.5 s, sys: 28.3 ms, total: 10.6 s
Wall time: 8.1 s


# optune

## catboost

In [17]:
def tuner(trial):    
    params = {
        "iterations": trial.suggest_int("iterations", 100, 500),
        "depth": trial.suggest_int("depth", 4, 9, step=1),
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-5, 2.5),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.1, 10.0),
        "od_wait": trial.suggest_int("od_wait", 10, 50),
    }    
    
    model = CatBoostClassifier(
        **params,
        random_seed=17,
        loss_function="Logloss",
        eval_metric="CrossEntropy",
        verbose=False,
        task_type="GPU" 
    )
    
    model.fit(
        df_train, 
        train_target.Target, 
        eval_set=[(df_test, test_target.Target)],
        early_stopping_rounds=500,
        verbose=False
    )
    
    test_prediction = model.predict_proba(df_test)[:, 1]
    
    return roc_auc_score(test_target.Target, test_prediction)


if CATBOOST_OPTUNA:
    study = optuna.create_study(direction='maximize', study_name='catboost_optuna')
    study.optimize(tuner, n_trials=200)
    print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

## xgboost

In [18]:
%%time
def tuner(trial):    
    params = {        
        "n_estimators": trial.suggest_int("n_estimators",100 ,1500),
        "max_depth":trial.suggest_int("max_depth", 4, 11, step=1),
        "min_child_weight": trial.suggest_int("min_child_weight", 3, 12,step=1),
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True), 
        "subsample": trial.suggest_float("subsample", 0.4, 1.0 ,step=0.1),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0 ,step=0.1),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-5, 1.5),
        "reg_lambda":  trial.suggest_float("reg_lambda", 1e-5, 2.5),
        "gamma": trial.suggest_float("gamma", 0.1, 1.0, step=0.1),
    }    
    
    model = xgboost.XGBRegressor(
        **params,
        random_state=17,
        n_jobs=-1, 
        objective="binary:logistic",
        eval_metric =roc_auc_score,
        tree_method="hist", 
        device = "cuda"
    )
    
    model.fit(        
        df_train, 
        train_target.Target.astype(int), 
        eval_set=[(df_test,test_target.Target.astype(int))],        
        verbose=False)
    
    test_prediction = model.predict(df_test)
    
    return roc_auc_score(test_target.Target.astype(int),test_prediction)

if XGBOOST_OPTUNA:
    
    
    study = optuna.create_study(direction='maximize', study_name='xgboost new feat')
    #study.optimize(tuner, n_trials=200) 
    study.optimize(tuner, n_trials=5)

    print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))


[I 2025-04-02 20:15:04,351] A new study created in memory with name: xgboost new feat
[I 2025-04-02 20:15:36,999] Trial 0 finished with value: 0.8512698664132213 and parameters: {'n_estimators': 1410, 'max_depth': 10, 'min_child_weight': 4, 'learning_rate': 0.00019232882538294308, 'subsample': 0.9, 'colsample_bytree': 0.30000000000000004, 'reg_alpha': 1.371831143046369, 'reg_lambda': 1.9739244547336712, 'gamma': 0.9}. Best is trial 0 with value: 0.8512698664132213.
[I 2025-04-02 20:15:44,703] Trial 1 finished with value: 0.849052220286182 and parameters: {'n_estimators': 324, 'max_depth': 9, 'min_child_weight': 7, 'learning_rate': 9.636616906554145e-06, 'subsample': 1.0, 'colsample_bytree': 0.30000000000000004, 'reg_alpha': 0.7558510422341848, 'reg_lambda': 2.110080681998944, 'gamma': 0.1}. Best is trial 0 with value: 0.8512698664132213.
[I 2025-04-02 20:15:49,622] Trial 2 finished with value: 0.8549870637309257 and parameters: {'n_estimators': 217, 'max_depth': 9, 'min_child_weight': 

Best trial: score 0.8549870637309257, params {'n_estimators': 217, 'max_depth': 9, 'min_child_weight': 9, 'learning_rate': 0.005660004346747614, 'subsample': 0.4, 'colsample_bytree': 0.4, 'reg_alpha': 1.0910653688731549, 'reg_lambda': 0.10780803660395323, 'gamma': 0.8}
CPU times: user 1min 17s, sys: 320 ms, total: 1min 18s
Wall time: 1min 7s


# end